In [ ]:
!pip install transformers bitsandbytes llama-stack --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.2/232.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

In [ ]:
import torch
print(torch.cuda.device_count())
print(torch.cuda.current_device())

# Set to GPU
torch.cuda.set_device("cuda:0")
print(torch.cuda.current_device())

In [ ]:
from google.colab import userdata
access_token = userdata.get("HF_TOKEN")
print(access_token)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.2-3B",
    device_map="auto",
    token=access_token
)

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B",
    device_map="auto",
    token=access_token
)

prompt = "Tell me about gravity"

# tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_auth_token=access_token)
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

output = model.generate(**model_inputs)
print(tokenizer.decode(output[0], skip_special_tokens=True))


In [ ]:
# Spoof data
prompts = [
        "Explain quantum computing in simple terms.",
        "What is the history of artificial intelligence?",
        "Describe the structure of DNA.",
        "What is the difference between Java and Python?",
        "Explain the concept of recursion in programming."
    ]

N_ITERATIONS = 3
MAX_NEW_TOKENS = 20

In [ ]:
import time
import numpy as np

def benchmark_test(benchmark_prompts):
  elapsed_times = np.array([])
  input_token_sizes = np.array([])
  output_token_sizes = np.array([])

  for it in range(N_ITERATIONS):
    for p in benchmark_prompts:
      with torch.no_grad():
        model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
        input_token_sizes = np.append(input_token_sizes,
                                      len(model_inputs['input_ids'][1]))

        # Measure inference time
        start_time = time.time()
        output = model.generate(**model_inputs, max_new_tokens=MAX_NEW_TOKENS,
                                pad_token_id=tokenizer.eos_token_id)
        end_time = time.time()

        output_token_sizes = np.append(output_token_sizes,
                                       len(output[0]))

      elapsed_t = end_time - start_time
      elapsed_times = np.append(elapsed_times, elapsed_t)
  return { "inference_times": elapsed_times,
           "input_token_sizes": input_token_sizes,
           "output_token_sizes": output_token_sizes }


In [ ]:
# Run benchmark in here
benchmark_results = benchmark_test(prompts)
print(f"Mean inference time (sec): {benchmark_results['inference_times'].mean()}")
print(f"Mean input token size: {benchmark_results['input_token_sizes'].mean()}")
print(f"Mean sec/token: { (benchmark_results['inference_times'] / benchmark_results['input_token_sizes']).mean() }")